In [7]:
import numpy as np
import random
from tqdm import tqdm

In [18]:
class board:
    def __init__(self, bInit):
        self.b=bInit
        
    def check(self, player):
        for i in range(3):
            if self.b[i,0]==player and self.b[i,1]==player and self.b[i,2]==player:
                return player
            if self.b[0,i]==player and self.b[1,i]==player and self.b[2,i]==player:
                return player
        if self.b[0,0]==player and self.b[1,1]==player and self.b[2,2]==player:
            return player
        if self.b[0,2]==player and self.b[1,1]==player and self.b[2,0]==player:
            return player
        if np.sum(np.where(self.b==0,1,0))==0:
            return 2
        return 0

    def toKey(self):
        s=0
        i=0
        for x in range(3):
            for y in range(3):
                s+=(self.b[x,y]+2)*10**i
                i+=1
        return int(s)

    def fromKey(self, i):
        self.b=np.zeros((3,3))
        for x in range(3):
            for y in range(3):
                self.b[x,y]=i%10 - 2
                i=int(i/10)
                
    def move(self, x, y, player):
        self.b[x,y]=player
        return self.check(player)
    
    def possibleActions(self):
        actions=[]
        for x in range(3):
            for y in range(3):
                if self.b[x,y]==0:
                    actions.append(x+3*y)
        return actions
    
    
def getAllStartStates():
    def f(l, b, i):
        x = i % 3
        y = int(i / 3)
        for p in [-1,0,1]:
            b.b[x,y]=p
            if np.sum(b.b==1)-np.sum(b.b==-1) in [0,1] and b.check(1)==0 and b.check(-1)==0:
                l.append(b.toKey())
            if i<8:
                f(l, b, i+1)
    
    b=board(np.zeros((3,3)))
    l = [b.toKey()]
    f(l,b,0)
    return l

In [55]:
def policyRandomMove(board, pure=False):
    p = np.where(board.b==0, 1, 0)
    if not pure:
        if p[1,1]==1:
            p[1,1]=3
        for x,y in [[0,0],[0,2],[2,0],[2,2]]:
            if p[x,y]==1:
                p[x,y]=2
    policy = p / np.sum(p)
    return chooseMove(policy)

def chooseMove(policy):
    u = np.random.uniform()
    c=0
    for x in [0,1,2]:
        for y in [0,1,2]:
            c+=policy[x,y]
            if u<c:
                return x,y


def policyMCMove(board, q, player):
    key=board.toKey()
    actions=board.possibleActions()
    best=q[key,actions[0]]
    bestAction=actions[0]
    for a in actions[1:]:
        if q[key,a]>best:
            bestAction=a
    x=bestAction % 3
    y=int(bestAction/3)
    return x,y

def policyMCMoveEpsGreedy(board, q, player, epsilon):
    if random.random()<epsilon:
        a = random.choice(board.possibleActions())
        x=a % 3
        y=int(a/3)
        return x,y
    else:
        return policyMCMove(board, q, player)
    
def playGame(q,epsilon=-1):
    nGames = 100000
    win1 =0
    winM1 = 0
    draw=0
    for _ in tqdm(range(nGames)):
        b=board(np.zeros((3,3)))
        player = random.choice([-1,1])
        gameOver=0
        while gameOver==0:
            if player==1:
                x,y=policyRandomMove(b)
            else:
                if epsilon>0:
                    x,y=policyMCMoveEpsGreedy(b,q,player,epsilon)
                else:
                    x,y=policyMCMove(b,q,player)                    
            gameOver = b.move(x,y,player)
            player=-player

        if gameOver==2:
            draw+=1
        elif gameOver==1:
            win1+=1
        else:
            winM1+=1


    print(win1/nGames, winM1/nGames, draw/nGames)

In [50]:
startStates=getAllStartStates()
q={}
c={}
n={}
for k in startStates:
    b=board(np.zeros((3,3)))
    b.fromKey(k)
    for a in b.possibleActions():
        q[(k,a)]=0.5
        n[(k,a)]=0
        c[(k,a)]=0

In [51]:
#epsilon greedy
epsilon=0.05
nGames = 100000
for _ in tqdm(range(nGames)):
    b=board(np.zeros((3,3)))
    player = random.choice([-1,1])
    gameOver=0
    listStateAction=[]
    while gameOver==0:
        if player==1:
            x,y=policyRandomMove(b)
        else:
            x,y=policyMCMoveEpsGreedy(b,q,player,epsilon)
            listStateAction.append((b.toKey(), (x, y)))
        gameOver = b.move(x,y,player)
        player=-player
        
    if gameOver==2:
        r=0.5
    elif gameOver==1:
        r=0
    else:
        r=1
    
    for k, (x,y) in listStateAction[::]:
        a=x+3*y
        q[(k,a)]=q[(k,a)]*n[(k,a)]+r
        n[(k,a)]+=1
        q[(k,a)]/=n[(k,a)]


playGame(q,epsilon)

100%|██████████| 100000/100000 [00:24<00:00, 4120.23it/s]

0.12792 0.81195 0.06013


In [48]:
startStates=getAllStartStates()
q={}
c={}
n={}
for k in startStates:
    b=board(np.zeros((3,3)))
    b.fromKey(k)
    for a in b.possibleActions():
        q[(k,a)]=0.5
        n[(k,a)]=0
        c[(k,a)]=0

In [49]:
#exploring starts
nGames = 100000
for _ in tqdm(range(nGames)):
    b=board(np.zeros((3,3)))
    player = -1
    randomAction=True
    gameOver=0
    listStateAction=[]
    while gameOver==0:
        if player==1:
            x,y=policyRandomMove(b)
        else:
            key=b.toKey()
            if randomAction:
                p = np.where(b.b==0, 1, 0)
                policy = p / np.sum(p)
                x,y = chooseMove(policy)
                randomAction=False
            else:
                x,y=policyMCMove(b,q,player)
            listStateAction.append((key, (x, y)))
        gameOver = b.move(x,y,player)
        player=-player
        
    if gameOver==2:
        r=0.5
    elif gameOver==1:
        r=0
    else:
        r=1
    
    for k, (x,y) in listStateAction[::]:
        a=x+3*y
        q[(k,a)]=q[(k,a)]*n[(k,a)]+r
        n[(k,a)]+=1
        q[(k,a)]/=n[(k,a)]

playGame(q)

100%|██████████| 100000/100000 [00:27<00:00, 3697.99it/s]

0.04568 0.8127 0.14162


In [58]:
startStates=getAllStartStates()
q={}
c={}
for k in startStates:
    b=board(np.zeros((3,3)))
    b.fromKey(k)
    for a in b.possibleActions():
        q[(k,a)]=0.5
        c[(k,a)]=0

In [59]:
#off policy
nGames = 1000000
for _ in tqdm(range(nGames)):
    b=board(np.zeros((3,3)))
    player = random.choice([-1,1])
    gameOver=0
    listStateAction=[]
    while gameOver==0:
        if player==1:
            x,y=policyRandomMove(b)
        else:
            x,y=policyRandomMove(b, True)
            listStateAction.append((b.toKey(), (x, y)))
        gameOver = b.move(x,y,player)
        player=-player
        
    if gameOver==2:
        r=0.5
    elif gameOver==1:
        r=0
    else:
        r=1

    W=1
    for k, (x,y) in listStateAction[::]:
        a=x+3*y
        c[(k,a)]=c[(k,a)]+W
        q[(k,a)]=q[(k,a)]+W*(r-q[(k,a)])/c[(k,a)]
        b=board(np.zeros((3,3)))
        b.fromKey(k)
        pA=b.possibleActions()
        bestA=pA[0]
        for aa in pA[1:]:
            if q[(k,aa)]>q[(k,bestA)]:
                bestA=aa
        if bestA!=a:
            break
        W=W*len(pA)

playGame(q)

100%|██████████| 100000/100000 [00:28<00:00, 3551.51it/s]

0.21706 0.70505 0.07789
